In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from scipy.stats import randint, uniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [11]:
df = pd.read_csv('Данные для построения модели, задача - Классификация - превышает ли значение CC50 медианное значение выборки.csv')
df

,BalabanJ,VSA_EState7,HallKierAlpha,Chi0n,SMR_VSA5,AvgIpc,SI,SPS,MolLogP,BCUT2D_LOGPHI,...,MolWt,PEOE_VSA8,TPSA,EState_VSA8,ExactMolWt,MaxAbsEStateIndex,BCUT2D_MWLOW,PEOE_VSA6,SMR_VSA10,Target Feature
0,1.164038,13.258223,-0.66,19.534409,105.750639,3.150503,28.125000,42.928571,7.1212,2.644698,...,384.652,35.342864,24.72,41.542423,384.350449,5.094096,9.700470,54.384066,11.423370,1
1,1.080362,14.135381,-0.08,19.794682,117.834321,3.150503,7.000000,45.214286,6.1556,2.658342,...,388.684,12.083682,24.06,52.176000,388.381750,3.961417,9.689226,54.384066,0.000000,0
2,1.219066,14.491619,-0.08,23.689110,117.834321,3.214947,0.720000,42.187500,7.1292,2.679014,...,446.808,23.671624,0.00,69.733111,446.458903,2.627117,9.681293,41.542423,0.000000,1
3,1.120513,14.695439,-0.66,20.241516,112.171461,3.179270,63.235294,41.862069,7.5113,2.644709,...,398.679,35.342864,24.72,41.542423,398.366099,5.097360,9.700497,60.804888,11.423370,1
4,1.136678,8.627311,-2.22,22.617677,86.488175,3.337074,1.300000,36.514286,9.1148,2.648473,...,466.713,22.253351,24.72,90.073360,466.334799,5.150510,9.700386,65.807891,22.798143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1.646946,7.985276,-1.65,18.825334,66.219879,3.023764,1.129017,49.133333,4.3002,2.739076,...,414.542,5.414990,69.67,33.770969,414.240624,12.934891,9.344314,38.841158,17.907916,0
997,1.535171,7.521836,-2.18,21.810933,79.185457,3.130958,1.062484,44.542857,3.8049,2.738755,...,485.621,10.829981,98.77,39.087758,485.277738,13.635345,9.343622,45.764895,23.815096,0
998,1.493776,10.055493,-1.83,23.633394,78.682541,3.204255,1.096761,41.973684,4.5381,2.738943,...,545.742,10.829981,98.77,39.087758,545.281109,13.991690,9.343613,45.764895,35.576981,0
999,2.325807,2.955837,-2.45,23.380977,66.219879,2.887043,1.031272,39.000000,3.3649,2.704027,...,522.635,11.835812,122.27,6.923737,522.282883,13.830180,9.364015,27.192033,29.660466,0


In [12]:
X, y = df.drop(['Target Feature'], axis=1), df['Target Feature']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
y.value_counts()

Target Feature
1    857
0    144
Name: count, dtype: int64

Классы не сбалансированны, произведем процесс балансировки классов

In [15]:
smote = SMOTE(sampling_strategy='auto', k_neighbors=10, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [16]:
X, y = X_resampled, y_resampled
y.value_counts()

Target Feature
1    857
0    857
Name: count, dtype: int64

Классы сбалансированны, значит процесс балансировки классов нам не требуется

In [17]:
class BestClassifierModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели классификации
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'F1-score',
            'value': 0.7
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LogisticRegression(max_iter=1000, random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': [0.01, 0.1, 1, 10],
                    'penalty': ['l1', 'l2'],
                    'solver': ['saga']
                },
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': KNeighborsClassifier(),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'n_neighbors': range(3, 15),
                    'weights': ['uniform', 'distance'],
                    'p': [1, 2]  # 1 = манхэттенское расстояние, 2 = евклидово расстояние
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestClassifier(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVC(random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            
            self.models[model]['metrics']['Accurac'] = accuracy_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Precision'] = precision_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Recall'] = recall_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['F1-score'] = f1_score(y, self.models[model]['data']['y_pred'])
            
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"Accuracy: {self.models[model]['metrics']['Accurac']:.4f}")
            print(f"Precision: {self.models[model]['metrics']['Precision']:.4f}")
            print(f"Recall: {self.models[model]['metrics']['Recall']:.4f}")
            print(f"F1-score: {self.models[model]['metrics']['F1-score']:.4f}")
            print()

            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return result

In [18]:
best_regression_model = BestClassifierModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LogisticRegression(max_iter=1000, random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...


g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_F

Лучшие параметры для модели LogisticRegression(max_iter=1000, random_state=42): {'C': 1, 'penalty': 'l1', 'solver': 'saga'}

Подбор гипперпараметров для модели KNeighborsClassifier() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели KNeighborsClassifier(): {'n_neighbors': 4, 'p': 1, 'weights': 'distance'}

Подбор гипперпараметров для модели RandomForestClassifier(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestClassifier(random_state=42): {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 63}

Подбор гипперпараметров для модели SVC(random_state=42) методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели SVC(ran

In [19]:
best_regression_model.get_best_model()

Обучаем модель LogisticRegression(max_iter=1000, random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LogisticRegression(max_iter=1000, random_state=42):
Accuracy: 0.8680
Precision: 0.8441
Recall: 0.8916
F1-score: 0.8672

Обучаем модель KNeighborsClassifier() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели KNeighborsClassifier():
Accuracy: 0.8932
Precision: 0.9217
Recall: 0.8514
F1-score: 0.8852

Обучаем модель RandomForestClassifier(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestClassifier(random_state=42):
Accuracy: 0.9670
Precisi

In [20]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=63,
                       random_state=42)
Предсказание сформировано


array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1]